<a href="https://colab.research.google.com/github/ttogle918/ds-section2-sprint2/blob/main/n224-model-selection/n224a-model-selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 4*

# 📝 Assignment
---

In [1]:
# !pip install category_encoders

In [2]:
# !pip uninstall pandas_profiling
# !pip install pandas_profiling
# !pip show pandas_profiling

In [3]:
# google 드라이브에 로컬 파일 업로드
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
target = 'vacc_h1n1_f'
train = pd.merge(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/train.csv"),
                 pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/train_labels.csv")[target],
                 left_index=True, right_index=True)  # train data + labels

test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/submission.csv")

In [25]:
train = train.dropna(axis=0)
train.head(5)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,agegrp,education_comp,raceeth4_i,sex_i,inc_pov,marital,rent_own_r,employment_status,census_region,census_msa,n_adult_r,household_children,n_people_r,employment_industry,employment_occupation,hhs_region,state,vacc_h1n1_f
3,2.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,Very Effective,Very Low,Not Very Worried,Very Effective,Very High,Not At All Worried,35 - 44 Years,4.0,1,2,1,1.0,1.0,Employed,3,"MSA, Not Principle City",2.0,2.0,4.0,"Professional, Scientific, and Technical Servic...",Legal Occupations,6,LOUISIANA,0
4,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Very Effective,Very Low,Not At All Worried,Not At All Effective,Very Low,Not At All Worried,55 - 64 Years,3.0,3,2,2,1.0,1.0,Employed,3,"MSA, Not Principle City",3.0,0.0,3.0,Education Services Industries,Office and Administrative Support Occupations,4,ALABAMA,1
5,3.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Very Effective,Somewhat Low,Not Very Worried,Very Effective,Somewhat High,Not Very Worried,45 - 54 Years,4.0,3,2,2,1.0,1.0,Employed,4,"MSA, Principle City",2.0,1.0,3.0,Other Services (except Public),Not ascertained,8,MONTANA,0
14,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Somewhat Effective,Somewhat Low,Not At All Worried,Somewhat Effective,Somewhat Low,Not At All Worried,65+ Years,1.0,4,2,2,2.0,2.0,Employed,4,"MSA, Principle City",1.0,0.0,1.0,"Arts, Entertainment, and Recreation Industries...",Building and Grounds Cleaning and Maintenance ...,9,NEVADA,0
15,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,Dont Know,Very Low,Not At All Worried,Dont Know,Very Low,Not At All Worried,35 - 44 Years,3.0,2,2,1,1.0,1.0,Employed,3,"MSA, Not Principle City",2.0,2.0,4.0,Retail Trade Industries,Sales and Related Occupations,6,LOUISIANA,0


In [27]:
import seaborn as sns
corr = train.corr()
corr['vacc_h1n1_f']
df_target_corr = corr[['vacc_h1n1_f']].sort_values('vacc_h1n1_f', ascending=False)
df_target_corr

,vacc_h1n1_f
vacc_h1n1_f,1.000000
doctor_recc_h1n1,0.372046
health_worker,0.246737
doctor_recc_seasonal,0.232959
h1n1_knowledge,0.197683
h1n1_concern,0.181707
health_insurance,0.123059
education_comp,0.113243
behavioral_wash_hands,0.101450
chronic_med_condition,0.100412


In [31]:
target = 'vacc_h1n1_f'

In [32]:
def engineer(df):
    """특성을 엔지니어링 하는 함수입니다."""
    
    # 높은 카디널리티를 가지는 특성을 제거합니다.
    selected_cols = df.select_dtypes(include=['number', 'object'])
    colnames = selected_cols.columns.tolist()
    labels = selected_cols.nunique()
    
    selected_features = labels[labels <= 30].index.tolist()
    df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    # 행동 // 1 : 예(좋음), 0:아니오(나쁨)
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    dels = [col for col in df.columns if ('behavioral_' in col )]
    df.drop(columns=dels, inplace=True)

    # seas : 계절독감 백신
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
    
    return df


train = engineer(train)
test = engineer(test)

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

train.head(2)

,h1n1_concern,h1n1_knowledge,doctor_recc_h1n1,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,agegrp,education_comp,raceeth4_i,sex_i,inc_pov,marital,rent_own_r,census_region,census_msa,n_adult_r,household_children,n_people_r,hhs_region,vacc_h1n1_f
0,2.0,2.0,0.0,1.0,0.0,1.0,0.0,Very Effective,Very Low,Not Very Worried,35 - 44 Years,4.0,1,2,1,1.0,1.0,3,"MSA, Not Principle City",2.0,2.0,4.0,6,0
1,1.0,2.0,0.0,0.0,0.0,1.0,0.0,Very Effective,Very Low,Not At All Worried,55 - 64 Years,3.0,3,2,2,1.0,1.0,3,"MSA, Not Principle City",3.0,0.0,3.0,4,1


In [30]:
import seaborn as sns
corr = train.corr()
corr['vacc_h1n1_f']
df_target_corr = corr[['vacc_h1n1_f']].sort_values('vacc_h1n1_f', ascending=False)
df_target_corr

,vacc_h1n1_f
vacc_h1n1_f,1.000000
doctor_recc_h1n1,0.372046
health_worker,0.246737
h1n1_knowledge,0.197683
h1n1_concern,0.181707
health_insurance,0.123059
education_comp,0.113243
chronic_med_condition,0.100412
child_under_6_months,0.067931
raceeth4_i,0.056919


In [8]:
from sklearn.model_selection import train_test_split

# train = pd.merge(train, train_labels[target], left_index=True, right_index=True)  # train data + labels
train, val = train_test_split(train, train_size=0.80, test_size=0.20, stratify=train[target], random_state=2) # divide
train.shape, val.shape, test.shape, submission.shape

((7456, 24), (1865, 24), (28104, 23), (28104, 2))

In [9]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
# submission = test[target]
features

Index(['h1n1_concern', 'h1n1_knowledge', 'chronic_med_condition',
       'child_under_6_months', 'health_insurance', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'agegrp', 'education_comp', 'raceeth4_i',
       'sex_i', 'inc_pov', 'marital', 'rent_own_r', 'census_region',
       'census_msa', 'n_adult_r', 'household_children', 'n_people_r',
       'hhs_region', 'Doctors'],
      dtype='object')

In [10]:
X_train.shape, y_train.shape, X_test.shape, submission.shape

((7456, 23), (7456,), (28104, 23), (28104, 2))

In [11]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train, minimal=True).to_notebook_iframe()

# 모델선택(Model Selection)

### 1) 캐글 대회를 이어서 진행합니다. RandomizedSearchCV 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.

- [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)를 사용하세요.
- 분류문제에서 맞는 [scoring parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values) metric을 사용하세요.
- [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) 사용을 권합니다.
- RandomizedSearchCV 를 사용해서 하이퍼파라미터 튜닝을 진행하고 최고 성능을 보이는 모델로 예측을 진행한 후 캐글에 제출합니다.
- **캐글 Leaderboard에서 개선된 본인 Score를 과제 제출폼에 제출하세요.**

In [12]:
from category_encoders import OrdinalEncoder
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [13]:
pipe = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(),
    RandomForestClassifier(random_state=42)
)

dists = {
    # 'ordinal_encoder__smoothing': [2.,20.,50.,60.,100.,500.,1000.], # int로 넣으면 error(bug)
    # 'ordinal_encoder__min_samples_leaf': randint(1, 10),     
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1) # max_features
}

clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=1,
    random_state = 42,
    n_jobs=-1
)

clf.fit(X_train, y_train);
print('최적 하이퍼파라미터: ', clf.best_params_)
print('MAE: ', -clf.best_score_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  4.3min finished


최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 0.18182496720710062, 'randomforestclassifier__n_estimators': 326, 'simpleimputer__strategy': 'mean'}
MAE:  0.1817313643189251


In [14]:
# rank_test_score: 테스트 순위
# mean_score_time: 예측에 걸리는 시간
pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').T

,4,1,32,12,16,35,46,15,39,43,7,23,37,11,34,44,42,19,14,6,41,45,26,25,47,22,10,36,5,31,28,20,29,24,30,21,40,8,13,17,33,2,3,38,0,18,49,48,27,9
mean_fit_time,2.23888,2.21308,2.5259,2.28384,2.71199,0.578159,2.19278,3.04612,4.23816,2.02507,3.34261,1.19352,7.61978,1.39873,2.76815,1.5826,2.62306,1.10483,4.8143,1.68352,2.67875,5.74858,2.58499,2.21838,0.346486,4.16434,4.61844,5.53825,4.88106,0.979683,7.25698,2.39535,3.96736,3.71343,4.84798,1.97252,1.88076,3.64374,9.53375,5.49902,0.91017,2.98222,10.3239,1.84338,3.4228,2.28407,0.950548,1.85177,1.84713,1.75107
std_fit_time,0.0101522,0.0394163,0.0513173,0.0206634,0.00864534,0.00766143,0.00833458,0.0135717,0.0146559,0.00486288,0.0358087,0.0138355,0.0355459,0.00999873,0.018384,0.000633424,0.0157792,0.0115859,0.0297539,0.0206508,0.0316349,0.0813454,0.00612742,0.00976826,0.000576218,0.0265439,0.0019925,0.00503075,0.0330791,0.0130406,0.0482734,0.0153625,0.0053552,0.0283203,0.0128251,0.0154584,0.0192501,0.0196152,0.0622631,0.00235521,0.00684756,0.0141831,0.048869,0.0263474,0.00696614,0.0389933,0.0370125,0.00972285,0.0123008,0.0155284
mean_score_time,0.2076,0.222353,0.156806,0.271407,0.165582,0.0614634,0.103776,0.159822,0.306333,0.162802,0.223105,0.0965475,0.308553,0.119388,0.230399,0.120504,0.163113,0.100354,0.347965,0.111513,0.144438,0.361122,0.126971,0.138856,0.0381134,0.19324,0.241759,0.245344,0.227491,0.0592708,0.306528,0.218507,0.204316,0.162278,0.19738,0.173564,0.1647,0.141948,0.337662,0.258982,0.0792484,0.124597,0.341593,0.0791242,0.124356,0.307066,0.0865629,0.254565,0.223662,0.223035
std_score_time,0.00125869,0.00372904,0.00568295,0.0138451,0.00175013,0.00107804,0.00224491,0.00570921,0.00827047,0.00192165,0.00836368,0.000985529,0.0109107,0.00280327,0.00319547,0.00152975,0.00261324,0.0069354,0.0188794,0.000710626,0.00486422,0.00498896,0.00486199,0.00215957,0.000895635,7.10072e-05,0.00501759,0.00588275,0.00371949,0.00082334,0.0075457,0.00939157,0.000823116,0.00691378,0.00674178,0.00158759,0.00403758,0.00320616,0.0120732,0.00605879,0.00120138,0.0111501,0.0157982,0.00278172,0.0014418,0.00264954,0.0111181,0.00743182,0.00942583,0.0171165
param_randomforestclassifier__max_depth,10,None,10,20,10,15,10,10,15,15,15,20,10,None,None,15,15,None,20,5,5,20,5,5,5,5,20,15,20,15,None,5,15,15,15,5,5,15,15,20,5,15,None,20,20,20,5,20,5,5
param_randomforestclassifier__max_features,0.181825,0.156019,0.439337,0.122038,0.449754,0.203061,0.712271,0.5677,0.324345,0.269412,0.382462,0.293488,0.80812,0.230894,0.249292,0.323203,0.417411,0.271349,0.311711,0.524775,0.818015,0.36363,0.91496,0.60596,0.266781,0.986887,0.563288,0.694785,0.611653,0.522733,0.729606,0.280935,0.561243,0.706857,0.760785,0.296274,0.272132,0.85994,0.90932,0.5979,0.314356,0.866176,0.96991,0.803672,0.950714,0.0452273,0.239562,0.0514788,0.0635584,0.0650516
param_randomforestclassifier__n_estimators,326,264,228,313,251,62,150,240,397,201,293,112,490,141,286,145,214,102,439,219,282,491,250,280,51,393,314,333,302,64,409,429,276,212,267,350,332,184,469,353,145,149,463,93,156,345,179,273,456,437
param_simpleimputer__strategy,mean,most_frequent,most_frequent,most_frequent,median,most_frequent,mean,median,mean,most_frequent,mean,most_frequent,mean,most_frequent,most_frequent,mean,mean,median,median,most_frequent,most_frequent,median,median,median,median,mean,median,median,mean,median,median,mean,mean,most_frequent,most_frequent,mean,median,mean,median,most_frequent,median,most_frequent,median,median,mean,mean,mean,mean,median,mean
params,"{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'random

In [15]:
# 기징 성능이 좋은 모델 불러오기|
pipe = clf.best_estimator_

In [16]:
from sklearn.metrics import mean_absolute_error
y_pred_train = pipe.predict(X_train)
y_pred_val = pipe.predict(X_val)
y_pred = pipe.predict(X_test)

In [17]:
submission

,Id,vacc_h1n1_f
0,0,0
1,1,1
2,2,1
3,3,1
4,4,0
...,...,...
28099,28099,0
28100,28100,1
28101,28101,1
28102,28102,0


In [18]:
from sklearn.metrics import f1_score
print('훈련 f1 score: ', f1_score(y_train, y_pred_train, average='binary'))
print('검증 f1 score: ', f1_score(y_val, y_pred_val, average='binary'))
mae = mean_absolute_error(submission['vacc_h1n1_f'], y_pred)
print(f'테스트세트 MAE: {mae}')
print('test f1 score: ', f1_score(submission['vacc_h1n1_f'], y_pred, average='binary'))

훈련 f1 score:  0.7651700680272109
검증 f1 score:  0.6240352811466372
테스트세트 MAE: 0.49832764019356673
test f1 score:  0.21597715949168672


In [19]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('MAE: ', -clf.best_score_)

최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 0.18182496720710062, 'randomforestclassifier__n_estimators': 326, 'simpleimputer__strategy': 'mean'}
MAE:  0.1817313643189251


In [22]:
submission['vacc_h1n1_f'] = y_pred
submission.to_csv('n224_zena_choi.csv', index=False)

## 🔥 도전과제


### 2) [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.
- 모델 성능을 높이기 위해 가능한 시도를 다 해보세요.
- 모델 성능에 가장 큰 영향을 준 하이퍼파라미터에 대해서 분석하고 설명해 보세요.



In [21]:
### 이곳에서 과제를 진행해 주세요 ### 